# Workforce Scheduling Benchmarks - Examples

This notebook demonstrates how to use the workforce scheduling benchmark framework.

**Author:** Ernest Owusu  
**Purpose:** PhD Application Portfolio

## Setup

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import our modules
from data_generator import generate_scheduling_problem, print_problem_summary
from greedy_scheduler import GreedyScheduler, print_solution_summary
from genetic_scheduler import GeneticScheduler

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Example 1: Generate a Scheduling Problem

In [ ]:
# Create a small problem instance
problem = generate_scheduling_problem(
    num_workers=15,
    num_shifts=8,
    num_days=7,
    seed=42
)

print_problem_summary(problem)

## Example 2: Solve with Greedy Algorithm

In [ ]:
# Initialize greedy scheduler
greedy = GreedyScheduler(priority_metric='cost')

# Solve
greedy_solution = greedy.solve(problem)

# Print results
print_solution_summary(greedy_solution, problem)

## Example 3: Solve with Genetic Algorithm

In [ ]:
# Initialize genetic algorithm
ga = GeneticScheduler(
    population_size=50,
    generations=30,
    crossover_rate=0.8,
    mutation_rate=0.15
)

# Solve
ga_solution = ga.solve(problem)

# Print results
print_solution_summary(ga_solution, problem)

## Example 4: Compare Algorithms

In [ ]:
# Compare different approaches
results = []

# Greedy with different priorities
for priority in ['cost', 'skills', 'balanced']:
    scheduler = GreedyScheduler(priority_metric=priority)
    solution = scheduler.solve(problem)
    results.append({
        'Algorithm': f'Greedy-{priority}',
        'Cost': solution.total_cost,
        'Coverage': solution.coverage_rate * 100,
        'Time': solution.solve_time
    })

# Genetic algorithm
results.append({
    'Algorithm': 'Genetic',
    'Cost': ga_solution.total_cost,
    'Coverage': ga_solution.coverage_rate * 100,
    'Time': ga_solution.solve_time
})

# Create comparison DataFrame
comparison_df = pd.DataFrame(results)
comparison_df

## Example 5: Visualize Comparison

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Cost comparison
axes[0].bar(comparison_df['Algorithm'], comparison_df['Cost'], color='skyblue', edgecolor='black')
axes[0].set_ylabel('Total Cost ($)', fontsize=12)
axes[0].set_title('Cost Comparison', fontsize=13, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Coverage comparison
axes[1].bar(comparison_df['Algorithm'], comparison_df['Coverage'], color='lightgreen', edgecolor='black')
axes[1].set_ylabel('Coverage Rate (%)', fontsize=12)
axes[1].set_title('Coverage Comparison', fontsize=13, fontweight='bold')
axes[1].set_ylim([90, 101])
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

# Time comparison
axes[2].bar(comparison_df['Algorithm'], comparison_df['Time'], color='salmon', edgecolor='black')
axes[2].set_ylabel('Solve Time (seconds)', fontsize=12)
axes[2].set_title('Time Comparison', fontsize=13, fontweight='bold')
axes[2].tick_params(axis='x', rotation=45)
axes[2].set_yscale('log')
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## Example 6: Analyze Worker Utilization

In [ ]:
# Analyze how work is distributed across workers
worker_assignments = {}

for assignment in greedy_solution.assignments:
    worker_id = assignment.worker_id
    worker_assignments[worker_id] = worker_assignments.get(worker_id, 0) + 1

# Create histogram
plt.figure(figsize=(10, 5))
plt.hist(worker_assignments.values(), bins=range(0, max(worker_assignments.values()) + 2), 
         color='steelblue', edgecolor='black', alpha=0.7)
plt.xlabel('Number of Shifts Assigned', fontsize=12)
plt.ylabel('Number of Workers', fontsize=12)
plt.title('Worker Utilization Distribution (Greedy Algorithm)', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.show()

print(f"Average shifts per worker: {np.mean(list(worker_assignments.values())):.2f}")
print(f"Std deviation: {np.std(list(worker_assignments.values())):.2f}")

## Example 7: Scalability Analysis

In [ ]:
# Test how algorithms scale with problem size
problem_sizes = [(10, 5, 7), (20, 10, 7), (30, 15, 7)]
scalability_results = []

for workers, shifts, days in problem_sizes:
    problem = generate_scheduling_problem(workers, shifts, days, seed=42)
    
    # Greedy
    greedy = GreedyScheduler()
    greedy_sol = greedy.solve(problem)
    
    scalability_results.append({
        'Size': f'{workers}W',
        'Algorithm': 'Greedy',
        'Time': greedy_sol.solve_time,
        'Cost': greedy_sol.total_cost
    })
    
    # Genetic
    ga = GeneticScheduler(population_size=50, generations=20)
    ga_sol = ga.solve(problem)
    
    scalability_results.append({
        'Size': f'{workers}W',
        'Algorithm': 'Genetic',
        'Time': ga_sol.solve_time,
        'Cost': ga_sol.total_cost
    })

scale_df = pd.DataFrame(scalability_results)

# Plot scalability
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for algo in ['Greedy', 'Genetic']:
    data = scale_df[scale_df['Algorithm'] == algo]
    axes[0].plot(data['Size'], data['Time'], marker='o', label=algo, linewidth=2)

axes[0].set_xlabel('Problem Size', fontsize=12)
axes[0].set_ylabel('Solve Time (seconds)', fontsize=12)
axes[0].set_title('Scalability: Solution Time', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

for algo in ['Greedy', 'Genetic']:
    data = scale_df[scale_df['Algorithm'] == algo]
    axes[1].plot(data['Size'], data['Cost'], marker='s', label=algo, linewidth=2)

axes[1].set_xlabel('Problem Size', fontsize=12)
axes[1].set_ylabel('Total Cost ($)', fontsize=12)
axes[1].set_title('Scalability: Solution Quality', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Conclusion

This notebook demonstrated:
1. Generating synthetic scheduling problems
2. Solving with greedy and genetic algorithms
3. Comparing algorithm performance
4. Analyzing solution quality and scalability

For more examples and full benchmarks, run `python benchmark.py` from command line.